# 合并ksrpc+frpc两阻塞代码于单Notebook

In [ ]:
def run_clear():
    print("启动 定时清屏")
    
    import time
    from IPython.display import clear_output
    
    try:
        while True:
            time.sleep(30)
            clear_output(wait=True)
    except KeyboardInterrupt:
        print("定时清屏 已中断")
    finally:
        print("定时清屏 服务已终止")
        
def run_frpc():
    print("启动 frpc 服务")
    
    import subprocess
    try:
        # 使用 subprocess 运行外部命令
        process = subprocess.Popen(
            ["./frpc", "-c", "./frpc.toml"],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            universal_newlines=True,
        )
        
        # 实时输出日志
        while True:
            output = process.stdout.readline()
            if output == '' and process.poll() is not None:
                break
            if output:
                print(output.strip())
                
        # 等待进程结束
        process.wait()
    except KeyboardInterrupt:
        print("frpc 已中断")
    except Exception as e:
        print("frpc 服务启动失败: ", e)
    finally:
        print("frpc 服务已终止")
        
def run_ksrpc():
    print("启动 ksrpc 服务")
    import multiprocessing
    import time

    from ksrpc import config
    config.PORT = 8000
    
    try:
        # 使用轮询机制检查退出标志
        from ksrpc.app import simple_
        simple_.main()
    except KeyboardInterrupt:
        print("ksrpc 已中断")
    except Exception as e:
        print("ksrpc 服务启动失败: ", e)
    finally:
        print("ksrpc 服务已终止")

In [ ]:
import multiprocessing
import time
from contextlib import contextmanager

@contextmanager
def process_manager(*processes):
    try:
        # 添加退出标志
        exit_flag = multiprocessing.Value('b', False)
        for p in processes:
            p.exit_flag = exit_flag  # 动态添加共享标志
            p.start()
        yield
    except KeyboardInterrupt:
        print("\n收到终止信号，关闭子进程...")
        exit_flag.value = True  # 通知子进程退出
        time.sleep(1)  # 给子进程时间处理
    finally:
        for p in processes:
            if p.is_alive():
                print(f"强制终止 {p.name}")
                p.terminate()
            p.join(timeout=1.0)
            if p.is_alive():
                print(f"警告: {p.name} 未能正常终止")

if __name__ == '__main__':
    # 创建进程时指定名称便于调试
    p1 = multiprocessing.Process(target=run_ksrpc, name="ksrpc")
    p2 = multiprocessing.Process(target=run_frpc, name="frpc")
    p3 = multiprocessing.Process(target=run_clear, name="clear")
    
    try:
        with process_manager(p1, p2, p3):
            # 主线程只需等待，由process_manager处理退出
            while True:
                time.sleep(2)
    except Exception as e:
        print(f"主进程错误: {e}")
    finally:
        print("所有服务已关闭")